# Convulation and Pooling
Library Import

In [54]:
import numpy as np
import tensorflow as tf
import keras
from keras import models
from keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator


Get the data

In [55]:
import zipfile

# Unzip the dataset
local_zip = 'validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('validation-horse-or-human')
zip_ref.close()

Devide the folder

In [56]:
import os

# Directory with our training horse pictures
train_horse_dir = os.path.join('./horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('./horse-or-human/humans')

# Directory with our training horse pictures
train_horse_dir = os.path.join('./validation-horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('./validation-horse-or-human/humans')

Callback Function

In [57]:
class callback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > 0.99:
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True

Convulation Layer

In [58]:
model = models.Sequential([
    # The first convolution
    layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(250, 250, 3)),
    layers.MaxPooling2D(2, 2),
    # The second convolution
    layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    layers.MaxPooling2D(3,3),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

model.compile(
    optimizer = tf.optimizers.Adam(),
    loss = 'binary_crossentropy',
    metrics=['accuracy'],
)

model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_24 (Conv2D)              │ (None, 250, 250, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 125, 125, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 41, 41, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 107584)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 256)            │    27,541,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,561,409 (105.14 MB)

 Trainable params: 27,561,409 (105.14 MB)

 Non-trainable params: 0 (0.00 B)

Preprocessing Image

In [59]:
train_dategen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_dategen.flow_from_directory(
    'horse-or-human',
    target_size=(250, 250),
    batch_size=128,
    class_mode='binary',
)

validation_generator = validation_datagen.flow_from_directory(
    'validation-horse-or-human',
    target_size=(250, 250),
    batch_size=32,
    class_mode='binary',
)



Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


Train the model

In [60]:
model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=15,
    verbose=1,
    validation_data = validation_generator,
    validation_steps=8,
)

Epoch 1/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 30s 3s/step - accuracy: 0.5162 - loss: 8.6610 - val_accuracy: 0.5000 - val_loss: 6.4836
Epoch 2/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6094 - loss: 1.7324 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - accuracy: 0.5622 - loss: 1.3677 - val_accuracy: 0.6758 - val_loss: 0.5546
Epoch 4/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7266 - loss: 0.3230 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - accuracy: 0.9045 - loss: 0.3702 - val_accuracy: 0.6523 - val_loss: 1.3798
Epoch 6/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9375 - loss: 0.1118 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - accuracy: 0.9085 - loss: 0.3157 - val_accuracy: 0.6680 - val_loss: 1.4691
Epoch 8/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9766 - loss: 0.0432 - val_accuracy: 0.0000e+00

Load prediction file

In [61]:
from tensorflow.keras.utils import load_img, img_to_array

img = load_img('predict/images.jpg', target_size=(300, 300))
x = img_to_array(img)
x /= 255
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images, batch_size=10)
print(classes[0])

if classes[0]>0.5:
    print(" is a human")
else:
    print(" is a horse")

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_20" is incompatible with the layer: expected axis -1 of input shape to have value 107584, but received input with shape (1, 160000)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 300, 300, 3), dtype=float32)
  • training=False
  • mask=None